In [ ]:
"""
【scrapy的使用】


创建scrapy 项目
scrapy startproject project_name 

创建爬虫
scrapy genspider spider_name 允许爬取的范围

运行爬虫 (进入到爬虫所有目录)
scrapy crawl spider_name


使用pipeline
1、实现pipelines.py中的 process_item 方法

2、去配置文件中开启pipeline
ITEM_PIPELINES = {
    'demo01.pipelines.Demo01Pipeline': 300, #数值越大数先执行
}


使用logging
去配置文件中配置

LOG_LEVEL = "WARNING"
LOG_FILE = "./file.log"	#保存路径，设置后不会再在终端中显示日志，需要使用logger


日志的级别有

CRITICAL - 严重错误
ERROR - 一般错误
WARNING - 警告信息
INFO - 一般信息
DEBUG - 调试信息  默认



import logging

logging.warning('your message') 输出格式: 2019-5-20 [root] WARNING:your message，不知道在哪个文件

logger = loggin.getLogger(__name__)
logger.warning('your message') 输出格式: 2019-5-20 [your file] WARNING:your message

# logging模块的设置可以通过loggin.baseConfig()来完成，百度一下参数即可


关于scrapy中实现翻页思路：
1、找到下一页的地址
2、构造一人下一页url地址的request请求传给调度器

next_url = response.xpath('pettern')
while len(next_url) > 0:
    yield scrapy.Request(next_url,callback=self.parse) # yield request对象，就可以交给scrapy引擎



【scrapy.Request知识点】
scrapy.Request(url,[,callback,method="GET",headers,body,cookie,meta,dont_filter=False])
注：中括号中的参数是可选参数
常用参数为：
callback：指定传入的url交给哪个解析函数去处理

meta：实现在不同的解析函数中传递数据，meta默认会携带部分信息，如下载延迟、请求深度等
别的解析函数中取meta
data = response.meta

注意：一般在解释函数中，通过都会有for循环去遍历，提取数据给item赋值，如果用meta参数将item交给下一个解释函数继续处理，而后面的解释函数
又有for循环时，很可能会导致item的数据错乱，或者重复，或者被覆盖了，因为scrapy的操作是【异步的】，后面解释函数在处理item时，前面解释函数
也在使用item这个引用，还有可能将引用指向了一个新的内存地址，这种情况，用meta传递item，就要注意使用深拷贝【copy.deepcopy()】





dont_filter：让scrapy的去重不会过滤当前的url,scrapy默认有url去重功能，对需要重复请求的url有重要用途

注意点：Request已经提供了cookie参数，cookie就不要再放到headers中去了


【pipeline】
pipeline中写入数据到mongodb，记得将item转为字典


【scrapy shell】
scrapy shell 是一个交互终端，我们可以在未启动spider的情况下尝试及调试代码，也可以用来测试xpath

用法：
scrapy shell url

response.url：当前响应的url地址
response.request.url：当前响应的请求的url地址
response.headers：响应头
response.body：响应体，也就是html代码，默认是byte类型
response.request.headers：当前响应的请求头


【setting相关】

settings中可以放一些全局变量供多个地方使用，一般全大写
1、spider 对象里面有settings属性，可直接使用
self.settings['key']
self.settings.get('key','default_value')

2、导入setting
from your_project import settings
print(settings.LOG_LEVEL)

ROBOTSTXT_OBEY = True #是否遵循robots.txt中的规则

CONCURRENT_REQUESTS # 并发数量

DOWNLOAD_DELAY # 下载延迟

COOKIES_ENABLED = True  # 请求是否带cookie

COOKIES_DEBUG = True # 开启cookie调试，会在调试信息中看到cookie的信息


【pipeline中的方法】

    def __init__(self, mongo_url, db_name):
        self.mongo_url = mongo_url
        self.db_name = db_name

    @classmethod
    def from_crawler(cls, crawler):
        return cls(mongo_url=crawler.settings.get("MONGO_URL"), db_name=crawler.settings.get("DB_NAME"))

    def open_spider(self, spider):
        self.client = MongoClient(self.mongo_url)
        self.db = self.client[self.db_name]
        self.context = self.db["users"]

    def close_spider(self, spider):
        self.client.close()

    def process_item(self, item, spider):
        self.context.update({'url_token': item['url_token']}, dict(item), True)
        return item




【scrapy中的CrawlSpider】 练手地址 http://www.cbrc.gov.cn/chinese/newIndex.html
简化：找查下一页url操作，构造自己的requests请求这一动作
满足某个条件的url地址，才发送给引擎，同时能够指定callback函数。

生成crawlspider的命令：
scrapy genspider -t crawl spider_name example.com

生成的爬虫代码基本一样的，但是spider的父类变成了 CrawlSpider ,多了一个 rules元组，少了parse方法
rules = (
    Rule(LinkExtractor(allow=r'正则'), callback='parse_item', follow=True),
)
LinkExtractor 链接提取器，提取url地址，即使url不完整，也不用管，框架会自动补全。
callback 提取出来的url地址的 response 会交给callback处理，不需要callback可以为None
follow 当前url地址的响应是否重新经过rules来提取url地址

注意：不能定义parse方法，有特殊意义。



【CrawlSpider补充】了解，需要用时回头看

LinkExtractor 更多常见参数：

allow：满足括号中‘正则表达式’的url会被提取，如果为人，则全部匹配。

deny：满足括号中的‘正则表达式’的url 不会 被提取，优先级高于allow。

allow_domains:会被提取的链接的domains。

deny_domains：不会被提取链接的domains。

restrict_xpaths：使用xpath表达式，和allow共同作用过滤链接，即xpath满足范围内的url
地址会被提取



spider.Rule常见参数：

link_extractor：是一个Link Extrack对象，用于定义需要提取的链接。

callback：从link_extractor中每获取到链接时，参数所指定的值作为回调函数。

follow：指定根据该规则从response提取的链接是否需要跟进，如果callback为None，
follow默认设置为True，否则默认为False。

process_links：指定该spider中哪个函数将会被调用，从link_extractor中获取到链接列表时
将会调用此函数，该方法主要用来过滤url。

process_request:指定该spider中哪个函数将会被调用，该规则提取到每个request时都会调用
该函数，用来过滤request。



【scrapy 模拟登录】
a、直接携带cooke
b、找到发送post请求的url地址，带信表单信息发送请求
c、重写父类的 start_resquests方法，自己构建post请求，把scrapy.Request的method参数填上POST


在start_requests方法中带cookie，不要放在headers中
def start_requests(self):
    cookies = {i.split("=")[0]:i.split("=")[1] for i in cookies_str.split("; ")}
    yield  scrapy.Request(selft.start_urls[0],callback=callback,cookies=cooikis)




【下载中间件】
与pipeline本样，需要定义一个类(middleware文件中)，然后在settings中开启


下载中件间的默认方法
process_request(self,request,spider):
    pass # 当每个request通过下载中间件时，该方法被调用
    

process_response(self,request,response,spider):
    pass # 当下载器完成http请求，传递响应给引擎的时候调用。
    # 这个方法一定要return
    # return request  表示这个请求经过引擎会交给调度器
    # return response 表示这个响应会经过引擎交给爬虫
    

下载中间件中，通过可以做反爬虫处理

class RandomUserAgent(object):
    def process_request(selff,request,spider):
        USER_AGENT = spider.settings.get('key')
        useragent = random.choice(USER_AGENT)
        request.headers.['User-Agent'] = useragent
        
        
        

class ProxyMiddleware(object):
    def process_request(self,request,spider):
        request.meta['proxy'] = "ip:port"  # 请求https的url，要用https的代理？







【scrapy post登录】
1、scrapy.Request 方法 method改成POST

2、scrapy.FromRequest 专门用来执行post提取的，可以带上表单、headers等参数


def parse(self, response):
    # 从start_urls 进来登录页面，可能需要提取一些tooken之类的表单参数
    post_data = {"account":"account","pwd":"pwd", ...} # 表单字典
    yield scrapy.FromRequest(login_url,formdata=post_data,call_back=self.after_login)
    
    
def after_login(selft,response):
    pass
    #post数据后的处理方法，不管登录是否成功



scrapy还能帮我们自动寻找form表单提取的url地址，


def parse(self,response):
    formdata = {表单域的名:对应的值,...}
    yield scrapy.FormRequest.from_response(response,formdata=formdata,callback=callback)




demo:

import scrapy

class GithubSpider(scrapy.Spider):
    name='github'
    allowed_domains = ['github.com']
    start_urls = ['https:github.com/login']

    def parse(self,response):
    yield scrapy.FormRequest.from_response(
        response, # 自动从response中寻找form表单
        formdata={'login':'your_account','password':'your_passworld'},
        callback=self.after_login
    )

    def after_login(self, response):
        pass


from_response 方法还有很多参数，可以传到定义看具体参数，方便scrapy准确找到正确的form表单




"""

In [ ]:
"""
【scrapy redis】 配置

SCHEDULER = "scrapy_redis.scheduler.Scheduler"
DUPEFILTER_CLASS = "scrapy_redis.dupefilter.RFPDupeFilter"
REDIS_URL = 'redis://192.168.3.8:6379' # 无密码

# 有密码
#REDIS_URL = 'redis://your_account:password@ip:6379'  

# 每次启动都清空队列
# SCHEDULER_FLUSH_ON_START =True

# 是否在爬取完成之后清空队列
# SCHEDULER_PERSIST = True


"""